<a href="https://colab.research.google.com/github/the-aryan-1712/phDOS-Prediction-using-ENN/blob/main/VibrationalEntropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from scipy.integrate import simpson
import numpy as np
import pandas as pd
import ast

In [ ]:
import numpy as np
import pandas as pd
import math
from scipy.integrate import simpson

# Constants
h = 6.62607015e-34  # (J·s)
k_B = 1.380649e-23  # (J/K)
T = 300  # Kelvin


n = h * 1e12 / (k_B * T)  # THz → Hz

# entropy integrand values
def calculate_entropy_integrand(freq_list, dos_list):
    result = []
    for freq, dos in zip(freq_list, dos_list):
        if freq > 0 and dos > 0:
            x = n * freq
            term = (x / (np.exp(x) - 1)) - np.log(1 - np.exp(-x))
            s = term * dos
            result.append(s)
        else:
            result.append(0.0)
    return result

def integrate_entropy(freq_list, integrand):
    return simpson(integrand, x=freq_list)


structure_csv = pd.read_csv('/content/structure_pred.csv')
structure_csv['normalised_phDOS_pred'] = structure_csv['normalised_phDOS_pred'].apply(ast.literal_eval)
structure_csv['Frequency (THz)'] = structure_csv['Frequency (THz)'].apply(ast.literal_eval)

# Vibrational entropy per atom (in units of k_B)
structure_csv['entropy_integrand'] = structure_csv.apply(
    lambda row: calculate_entropy_integrand(row['Frequency (THz)'], row['normalised_phDOS_pred']),
    axis=1
)

structure_csv['vibrational_entropy_kB'] = structure_csv.apply(
    lambda row: integrate_entropy(row['Frequency (THz)'], row['entropy_integrand']),
    axis=1
)
del structure_csv['entropy_integrand']

structure_csv.to_csv('structure_pred_entropy.csv', index=False)


np.float64(3.0)